## Problem

4 つの原料 (w, x, y, z) を使って 4 つの製品 (a, b, c, d) を生産します。

製品 1 kg を作るのに必要な原材料、一日に使用できる原料の量 (kg)、製品の利益 (万円) が下表のように定義されています。

一日の利益が最大となるように、各製品の生産量を決めてください。

|        | 原料w | 原料x | 原料y | 原料z | 利益 |
|     -  |     - |     - |     - |     - |    - |
| 製品A  |     2 |     1 |     0 |     0 |    5 |
| 製品B  |     0 |     2 |     1 |     0 |    3 |
| 製品C  |     0 |     0 |     1 |     2 |    2 |
| 製品D  |     1 |     0 |     0 |     2 |    4 |
| 使用量 |     4 |     8 |     6 |    10 |      |

(http://www.nct9.ne.jp/m_hiroi/light/pulp01.html#chap03)

In [1]:
import pulp
import pandas as pd

In [2]:
df = pd.DataFrame(
    columns=["原料w", "原料x", "原料y", "原料z", "利益"],
    index=["製品A", "製品B", "製品C", "製品D", "使用量"],
    data=[
        [2, 1, 0, 0, 5],
        [0, 2, 1, 0, 3],
        [0, 0, 1, 2, 2],
        [1, 0, 0, 2, 4],
        [4, 8, 6, 10],
        ]
    )
df

,原料w,原料x,原料y,原料z,利益
製品A,2,1,0,0,5.0
製品B,0,2,1,0,3.0
製品C,0,0,1,2,2.0
製品D,1,0,0,2,4.0
使用量,4,8,6,10,NaN


In [3]:
problem = pulp.LpProblem(name="production_planning", sense=pulp.LpMaximize)
problem

production_planning:
MAXIMIZE
None
VARIABLES

In [4]:
products = pulp.LpVariable.dicts("", df.index[:-1], lowBound=0, cat=pulp.LpInteger)
products

{'製品A': _製品A, '製品B': _製品B, '製品C': _製品C, '製品D': _製品D}

In [5]:
problem += pulp.lpSum([products[i] * df["利益"][i] for i in products.keys()])
problem

production_planning:
MAXIMIZE
5.0*_製品A + 3.0*_製品B + 2.0*_製品C + 4.0*_製品D + 0.0
VARIABLES
0 <= _製品A Integer
0 <= _製品B Integer
0 <= _製品C Integer
0 <= _製品D Integer

In [6]:
for j in df.columns[:-1]:
    problem += pulp.lpSum([products[i] * df[j][i] for i in products.keys()]) <= df[j]["使用量"]
problem

production_planning:
MAXIMIZE
5.0*_製品A + 3.0*_製品B + 2.0*_製品C + 4.0*_製品D + 0.0
SUBJECT TO
_C1: 2 _製品A + _製品D <= 4

_C2: _製品A + 2 _製品B <= 8

_C3: _製品B + _製品C <= 6

_C4: 2 _製品C + 2 _製品D <= 10

VARIABLES
0 <= _製品A Integer
0 <= _製品B Integer
0 <= _製品C Integer
0 <= _製品D Integer

In [7]:
result = problem.solve()
print("Status", pulp.LpStatus[result])

Status Optimal


In [8]:
print("=" * 3, "Result", "=" * 29)
[print(i, ":", products[i].value() ) for i in products.keys()]
print('総利益: {}'.format(pulp.value(problem.objective)))
print("=" * 40)

=== Result =============================
製品A : 0.0
製品B : 4.0
製品C : 1.0
製品D : 4.0
総利益: 30.0
